In [1]:
import glob,os
import rasterio as rio
import geopandas as gpd
import numpy as np
from rasterio.plot import reshape_as_image
from sklearn.preprocessing import MinMaxScaler

training_path = "Input/sentinel/test_data_from_drive/patches_all/test/"
training_normalised_path = "Input/sentinel/test_data_from_drive/patches_all/normalised_test/"
target_file_path = "Input/Target/concat/target_yield.shp"
patch_dim = (256, 256, 12)


In [2]:
training_file_list = glob.glob(os.path.join(training_path,"*.tif"))
target_gdf = gpd.read_file(target_file_path)
print("Total Number of Patches:",len(training_file_list))
ignore_patch_list = list()
count = 0
for file in training_file_list:
    patch_src = rio.open(file)

    patch_src_read_norm = list()
    f_name = file.split("/")[-1].split(".")[0]
    patch_src_read = reshape_as_image(patch_src.read()[0:12]) ## Change the index here to add or remove the mask layer
    # print(0)
    if patch_src_read.shape != patch_dim:
        ignore_patch_list.append(f_name)
        # print("Patch Dimensions Mismatch, skipping patch : {}".format(f_name))
        continue
    # print(1)
    if np.isnan(patch_src_read).any():
        # print("Has Nan values, skipping patch : {}".format(f_name))
        continue
    # print(2)
    query = target_gdf.query(f"patch_name == '{f_name}'")["ykg_by_e7"]
    if len(query) != 1:
        # print("patch has no target value, skipping patch : {}".format(f_name))
        continue
    
    for band_no in range(0,12):
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(patch_src_read[:,:,band_no])
        patch_src_read_norm.append(scaler.transform(patch_src_read[:,:,band_no]))
    patch_src_read_norm.append(patch_src.read()[12,:,:])
    patch_src_read_norm = np.array(patch_src_read_norm)
    # print(patch_src_read_norm.shape)
    # print("Minimum: ",patch_src_read_norm.min())
    # print("Maximum: ",patch_src_read_norm.max())
    patch_src.close()
    # print(count)
    outpath = training_normalised_path+"/"+f_name+".tif"
    meta = patch_src.meta.copy()
    with rio.open(outpath, 'w', **meta) as outds:
        outds.write(patch_src_read_norm)
    count +=1
    # if count > 5:
    #     break

# # self.y = self.scaler.fit_transform(np.array(self.y).reshape(-1, 1))
# y = np.array(y)
# y = np.expand_dims(y,-1)
# x = np.array(x)

# x = (x-np.min(x))/(np.max(x)-np.min(x))
# print("Any Null values? ",np.isnan(x).any())
# # print(self.y)
# # self.x = np.nan_to_num(self.x, nan=0)# Check for different value for no data
# print(f"x shape :{x.shape}, y shape: {y.shape}")
# # print(np.nanmin(self.x),np.nanmax(self.x))
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
# return X_train, X_test, y_train, y_test
# #Also, split the training into train and val
# # For testing, keep a part of the dataset as seperate (final month)

# X_train, X_test, y_train, y_test = read_training()

Total Number of Patches: 2677
